In [17]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.callbacks import get_openai_callback
from tiktoken import get_encoding
from os import getenv, path
from git import Repo

In [2]:
# Retrieving an API key from environmental variable
api_key = getenv('API')
if api_key is None:
    raise ValueError("Environmental variable `API` doesn't contain the API key!")

In [3]:
# Cloning the repository
repo_path = "./repo"
if not path.exists(repo_path):
    repo = Repo.clone_from("https://github.com/staszczm/AIChallange-A.I.diots", to_path=repo_path)

In [4]:
# Loading the repository
loader = GenericLoader.from_filesystem(
    path=repo_path,
    glob="**/*",  # Recursively find all files within given repo
    suffixes=[".py"],  # Search for Python code files
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=250),
)
documents = loader.load()

In [5]:
# Splitting the files into smaller chunks
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)

In [25]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

tokens = 0
for text in texts:
    tokens += num_tokens_from_string(str(text), "cl100k_base") 
print("Tokens: " + str(tokens))

price = tokens * 0.0004 / 1000
print("Price: " + str(price))

Tokens: 20440
Price: 0.008176000000000001


In [6]:
# Adding split files into embedding database and defining what it returns
db = Chroma.from_documents(texts, OpenAIEmbeddings(openai_api_key=api_key))
retriever = db.as_retriever(
    search_type="mmr",  # Maximal Marginal Relevance
    search_kwargs={"k": 10},  # Return the top 10 matches
)

In [7]:
# Initialize LangChain with the OpenAI model
lc = ChatOpenAI(
    model_name='gpt-4-1106-preview',  # Alternative options ['gpt-4-1106-preview', 'gpt-3.5-turbo-1106']
    openai_api_key = api_key,
)   

In [8]:
# Memorising the context of previous messages
memory = ConversationSummaryMemory(
    llm=lc, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(lc, retriever=retriever, memory=memory)

In [9]:
# Example prompt
with get_openai_callback() as cb:
    question = "What is this project about?"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

I don't know the specific nature and objectives of the project beyond what the provided context suggests. However, based on the code snippets, it seems to be a Django-based web application related to an educational institution, where various models and views are defined to handle operations around academic sessions, terms, students, classes, subjects, and financial transactions like invoices and receipts.

Here are some inferred functionalities from the provided context:

1. **Academic Session and Term Management**: The application allows for the creation and management of academic sessions and terms, including setting a current session and term.

2. **Student Management**: It includes features to list, view details, create, and update student records. There is a mention of a `StudentBulkUpload`, which suggests functionality to upload multiple student records at once, although the details of this functionality are not provided.

3. **Class and Subject Management**: The system allows fo

In [10]:
# Example prompt
with get_openai_callback() as cb:
    question = "How can I initialize a React agent?"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

I don't have information on setting up a React agent from scratch in the provided context. However, I can explain the general steps to create a new React project from scratch:

1. **Install Node.js and npm**: Before you can start working with React, you must have Node.js and npm (Node Package Manager) installed on your system. You can download them from [nodejs.org](https://nodejs.org/).

2. **Create a New React App**: Use the `create-react-app` command-line utility which is the officially recommended way to create new React applications. To create a new app, run the following command in your terminal:

   ```bash
   npx create-react-app my-react-app
   ```

   Replace `my-react-app` with whatever you want to name your application.

3. **Navigate to Your App Directory**: Once the installation is complete, navigate to your newly created app directory:

   ```bash
   cd my-react-app
   ```

4. **Start the Development Server**: Start the local development server to see your app:

   ```ba

In [11]:
# Example prompt
with get_openai_callback() as cb:
    question = "Describe in details just the pieces of code that cover Django"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

Certainly! The Django code segments provided are fragments from an educational management web application. I'll describe each code segment and its purpose in the context of the application.

### Models
Models in Django are used to define the data structure. The models provided (`SiteConfig`, `AcademicSession`, `AcademicTerm`, `Subject`, `StudentClass`, `Student`, and `Invoice`) represent the various entities that are part of the educational management system.

- `SiteConfig`: This model appears to store key-value pairs for site configuration settings.
- `AcademicSession`: Represents an academic year or period.
- `AcademicTerm`: Represents a term or semester within an `AcademicSession`.
- `Subject`: Represents a subject or course taught in the school.
- `StudentClass`: Represents the different classes or grades in the school.
- `Student`: Not shown in the provided code, but it would typically represent the students.
- `Invoice`: Not shown in the provided code, but likely represents bill

In [12]:
# Example prompt
with get_openai_callback() as cb:
    question = "Do you have any idea on how to improve this project?"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

Based on the provided context, the Django educational management web application could be improved in several ways. Here are some suggestions:

1. **User Experience and Interface Enhancements**:
   - Improve the UI/UX with a modern, responsive design to ensure usability across various devices and screen sizes.
   - Implement AJAX-based form submissions and page updates for a smoother user experience.

2. **Security Updates**:
   - Ensure regular updates to Django and its dependencies to patch vulnerabilities.
   - Implement more robust user input validation to prevent SQL injection and XSS attacks.
   - Use HTTPS to secure data in transit.

3. **Performance Optimization**:
   - Optimize queries to reduce database load and speed up page loading times.
   - Implement caching for static assets and frequently accessed data.
   - Use a CDN for serving static files to improve load times for users at different geographical locations.

4. **Feature Expansion**:
   - Integrate a more comprehens

In [13]:
# Example prompt
with get_openai_callback() as cb: 
    question = "Describe in details what file models.py contains"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

The `models.py` file in the context of a Django-based educational management web application contains definitions for the database models which represent the various entities involved in the application. Based on the provided context, the models represent configurations, academic sessions, terms, subjects, classes, and students. Here's a breakdown of each model:

1. `SiteConfig`: This model is designed to store site configuration settings with a `key` and `value` field. Each configuration setting is represented by a unique slug in the `key` field and its corresponding value in the `value` field.

2. `AcademicSession`: This model represents an academic session (like 2020/2021). It has a `name` field, which must be unique, and a `current` boolean field to indicate whether this session is the current one. The `ordering` Meta class option orders records by descending session names.

3. `AcademicTerm`: This model represents an academic term (like First Term, Second Term, etc.). It has a `na

In [14]:
# Example prompt
with get_openai_callback() as cb:
    question = "Can you generate some Python code of new functionality that could be possibly added to this project?"
    result = qa(question)
    print(f"{result['answer']}")
    print(cb)

Certainly! Since the provided context already includes models like `Student`, `AcademicSession`, `AcademicTerm`, and `Subject`, along with some views for listing these records, we can enhance the functionality by adding a feature to calculate and record students' grades for different subjects in each term.

First, we need to create a new model to store the grades, then we'll write a view to input the grades, and finally, we'll create a template to enter the grades through the web interface.

1. Define a `Grade` model in `models.py`:

```python
class Grade(models.Model):
    student = models.ForeignKey(Student, on_delete=models.CASCADE)
    subject = models.ForeignKey(Subject, on_delete=models.CASCADE)
    academic_term = models.ForeignKey(AcademicTerm, on_delete=models.CASCADE)
    score = models.DecimalField(max_digits=5, decimal_places=2)

    class Meta:
        unique_together = ('student', 'subject', 'academic_term')

    def __str__(self):
        return f"{self.student} - {self.